In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import numpy as np
from nltk.corpus import stopwords 
stop = stopwords.words('english')

In [ ]:
train = pd.read_csv("../Class11/Datasets/reviews.csv")

In [ ]:
train.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
train = train[['Summary','Text']] 

In [ ]:
train['text_length'] = train['Text'].str.count(' ') #train er vitor kto gulo string ase ta count korbo

In [ ]:
train['text_length'].describe() #describe er bapper ta box plot related.

count    568454.000000
mean         81.005522
std          80.807102
min           2.000000
25%          33.000000
50%          57.000000
75%          99.000000
max        3525.000000
Name: text_length, dtype: float64

In [ ]:
train['summary_length'] = train['Summary'].str.count(' ')
train['summary_length'].describe()

count    568427.000000
mean          3.128462
std           2.619420
min           0.000000
25%           1.000000
50%           3.000000
75%           4.000000
max          41.000000
Name: summary_length, dtype: float64

In [ ]:
train = train.loc[train['summary_length']<8] #<8-->summary_length  er  word jodi 8 er cheye choto hoy tahole oi data nia kaj korbe 

In [ ]:
train = train.loc[train['text_length']<30] #<30-->summary_length  er  word jodi 30 er cheye choto hoy tahole oi data nia kaj korbe 

In [ ]:
#iloc=index location
#loc=location
#'' --> space nai
#' '---->space ase

In [ ]:
print(train.shape)
print(train.head())

(109053, 4)
                   Summary                                               Text  \
4              Great taffy  Great taffy at a great price.  There was a wid...   
7   Wonderful, tasty taffy  This taffy is so good.  It is very soft and ch...   
8               Yay Barley  Right now I'm mostly just sprouting this so my...   
9         Healthy Dog Food  This is a very healthy dog food. Good for thei...   
13       fresh and greasy!  good flavor! these came securely packed... the...   

    text_length  summary_length  
4            29             1.0  
7            27             2.0  
8            25             1.0  
9            24             2.0  
13           14             2.0  


In [ ]:
train['text_lower'] = train['Text'].str.lower() #small letter e korlm/ str---> nilam karon amra sudhu string gulo nia kaj korbo.
train['text_no_punctuation'] = train['text_lower'].str.replace('[^\w\s]','') #[^\w\s]----->white space

/opt/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
train['summary_lower'] = train["Summary"].str.lower()
train['summary_no_punctuation'] =  '_start_' + ' ' +train['summary_lower'].str.replace('[^\w\s]','')+ ' ' +'_end_' #_start_/'_end_--->protita summary er ses e blank space thkbe ,

/opt/anaconda3/envs/DL/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
max_features1 = 5000 #sudhu main word/token gulo nibe  jar poriman hbe 5000
maxlen1 = 30 #maximum lenth 30 hbe (text_length)

max_features2 = 5000 
maxlen2 = 8 ##maximum lenth 8 hbe (summary_length)

In [ ]:
tok1 = tf.keras.preprocessing.text.Tokenizer(num_words=max_features1) 
tok1.fit_on_texts(list(train['text_no_punctuation'].astype(str))) #fit to cleaned text(text_no_punctuation)kno punction thkbe na ,even number thkle ta string e conv. hbe 
tf_train_text =tok1.texts_to_sequences(list(train['text_no_punctuation'].astype(str))) #astype(str)---> ja ase sob str e conv hbe
tf_train_text =tf.keras.preprocessing.sequence.pad_sequences(tf_train_text, maxlen=maxlen1) #let's execute pad step //

In [ ]:
tok2 = tf.keras.preprocessing.text.Tokenizer(num_words=max_features2, filters = '*') #------>
tok2.fit_on_texts(list(train['summary_no_punctuation'].astype(str))) #fit to cleaned text
tf_train_summary = tok2.texts_to_sequences(list(train['summary_no_punctuation'].astype(str)))
tf_train_summary = tf.keras.preprocessing.sequence.pad_sequences(tf_train_summary, maxlen=maxlen2, padding ='post')#padding ='post'---->layer gula change hbe,sme hole change hto na//pad_sequences--->RNN vlo vbe kaj korbe

# Define Model Architecture

In [ ]:
#vectorized

vectorized_summary = tf_train_summary  #single array te convert kora/machine related e conv kora
decoder_input_data = vectorized_summary[:, :-1]
decoder_target_data = vectorized_summary[:, 1:]
#summary main bkz oi ta dia kaj hbe tai oi ta decode korlm

vectorized_text = tf_train_text
encoder_input_data = vectorized_text #eta decode na korleo cholbe
doc_length = encoder_input_data.shape[1] #shape[1]--->index ke access korteci/1 number sari ta access korteci

In [ ]:
vocab_size_encoder = len(tok1.word_index) + 1 #last indes jate miss na hoy ,ti 1 jog korlm
vocab_size_decoder = len(tok2.word_index) + 1

### Define Model Architecture

In [ ]:
#arbitrarly set latent dimension for embedding and hidden units
latent_dim = 50

In [ ]:
encoder_inputs = tf.keras.Input(shape=(doc_length,), name='Encoder-Input') #shape=(doc_length)----> vectorized value (18 number columns)

x = tf.keras.layers.Embedding(vocab_size_encoder, #(vocab_size_encoder)---->#Encoding value (19 number columns)
                              latent_dim, name='Body-Word-Embedding',#miss kisu thkle ta fillup korbe/latent_dim--->(20 number columns)
                              mask_zero=False)(encoder_inputs) #mask_zero=False---->kno index er value jdi missing thake ta fill up korbo na

x = tf.keras.layers.BatchNormalization(name='Encoder-Batchnorm-1')(x)

_, state_h = tf.keras.layers.GRU(latent_dim, #_,---> output ta skip korteci but state_h dekbo ti nisi
                                 return_state=True, 
                                 name='Encoder-Last-GRU')(x)

encoder_model = tf.keras.Model(inputs=encoder_inputs,
                               outputs=state_h, 
                               name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs) #encoder_model(encoder_inputs)--> ta sequence hesabe nilm


decoder_inputs = tf.keras.Input(shape=(None,),
                                name='Decoder-Input')  # for teacher forcing

dec_emb = tf.keras.layers.Embedding(vocab_size_decoder, #Encoding value (19 number columns)
                                    latent_dim, #latent_dim--->(20 number columns)
                                    name='Decoder-Word-Embedding',
                                    mask_zero=False)(decoder_inputs)

dec_bn = tf.keras.layers.BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)


decoder_gru = tf.keras.layers.GRU(latent_dim, 
                                  return_state=True, 
                                  return_sequences=True, 
                                  name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, 
                                    initial_state=seq2seq_encoder_out) 
x = tf.keras.layers.BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)


decoder_dense = tf.keras.layers.Dense(vocab_size_decoder,
                                      activation='softmax',
                                      name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)

seq2seq_Model = tf.keras.Model([encoder_inputs
                                , decoder_inputs], decoder_outputs)

seq2seq_Model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
                      loss='sparse_categorical_crossentropy')

** Examine Model Architecture Summary **

In [ ]:
seq2seq_Model.summary()

# Train Model

In [ ]:
batch_size = 64
epochs = 2 
history = seq2seq_Model.fit([encoder_input_data,
                             decoder_input_data], 
                            np.expand_dims(
                                decoder_target_data, -1),
          batch_size=batch_size,  epochs=epochs ,  
                            validation_split=0.12) 

Epoch 1/2
1500/1500 [==============================] - 121s 78ms/step - loss: 2.6445 - val_loss: 1.9713
Epoch 2/2
1500/1500 [==============================] - 114s 76ms/step - loss: 1.7928 - val_loss: 1.7531


In [ ]:
test_text = ['this product is great  gives you so much energy and tastes great  try this cafe  latte and all the other flavors and you will not be disappointed']

In [ ]:
tok1.fit_on_texts(test_text)

In [ ]:
raw_tokenized = tok1.texts_to_sequences(test_text)
raw_tokenized = tf.keras.preprocessing.sequence.pad_sequences(raw_tokenized, maxlen=maxlen1)

In [ ]:
body_encoding = encoder_model.predict(raw_tokenized) #predict the encoder state of the new sentence

In [ ]:
latent_dim = seq2seq_Model.get_layer('Decoder-Word-Embedding').output_shape[-1]

In [ ]:
#remember the get layer methodo for getting the embedding (word clusters)
decoder_inputs = seq2seq_Model.get_layer('Decoder-Input').input 
dec_emb = seq2seq_Model.get_layer('Decoder-Word-Embedding')(decoder_inputs)
dec_bn = seq2seq_Model.get_layer('Decoder-Batchnorm-1')(dec_emb)


In [ ]:
gru_inference_state_input = tf.keras.Input(shape=(latent_dim,),
                                           name='hidden_state_input')



In [ ]:
gru_out, gru_state_out = seq2seq_Model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input])

In [ ]:
# Reconstruct dense layers
dec_bn2 = seq2seq_Model.get_layer('Decoder-Batchnorm-2')(gru_out)
dense_out = seq2seq_Model.get_layer('Final-Output-Dense')(dec_bn2)

In [ ]:
decoder_model = tf.keras.Model([decoder_inputs, 
                                gru_inference_state_input],
                          [dense_out, gru_state_out])

In [ ]:
original_body_encoding = body_encoding

In [ ]:
state_value = np.array(tok2.word_index['_start_']).reshape(1, 1)

In [ ]:
state_value

array([[1]])

In [ ]:
decoded_sentence = []
stop_condition = False

In [ ]:
vocabulary_inv = dict((v, k) for k, v in tok2.word_index.items())

In [ ]:
#print(vocabulary_inv)

In [ ]:
while not stop_condition:
    #print(1)
    preds, st = decoder_model.predict([state_value, body_encoding])

    pred_idx = np.argmax(preds[:, :, 2:]) + 2
    pred_word_str = vocabulary_inv[pred_idx]
    print(pred_word_str)
    if pred_word_str == '_end_' or len(decoded_sentence) >= maxlen2:
        stop_condition = True
        break
    decoded_sentence.append(pred_word_str)

    # update the decoder for the next word
    body_encoding = st
    state_value = np.array(pred_idx).reshape(1, 1)
    #print(state_value)

great
product
_end_


In [ ]:
train.tail()

,Summary,Text,text_length,summary_length,text_lower,text_no_punctuation,summary_lower,summary_no_punctuation
568439,a-ok,We need this for a recipe my wife is intereste...,22,0.0,we need this for a recipe my wife is intereste...,we need this for a recipe my wife is intereste...,a-ok,_start_ aok _end_
568442,Great Cafe Latte,This product is great. Gives you so much ener...,28,2.0,this product is great. gives you so much ener...,this product is great gives you so much energ...,great cafe latte,_start_ great cafe latte _end_
568448,Very large ground spice jars.,My only complaint is that there's so much of i...,28,4.0,my only complaint is that there's so much of i...,my only complaint is that theres so much of it...,very large ground spice jars.,_start_ very large ground spice jars _end_
568449,Will not do without,Great for sesame chicken..this is a good if no...,25,3.0,great for sesame chicken..this is a good if no...,great for sesame chickenthis is a good if not ...,will not do without,_start_ will not do without _end_
568453,Great Honey,"I am very satisfied ,product is as advertised,...",20,1.0,"i am very satisfied ,product is as advertised,...",i am very satisfied product is as advertised i...,great honey,_start_ great honey _end_


In [ ]:
train['summary_no_punctuation'][568442]

'_start_ great cafe latte _end_'

In [ ]:
train['Text'][568442]

'This product is great.  Gives you so much energy and tastes great.  Try this Cafe Latte and all the other flavors and you will not be disappointed.'